# Design Patterns with First-Class Funcitons

## Case Study: Refactoring Strategy

### Classic Strategy

[전략 디자인 패턴 UML class diagram](https://upload.wikimedia.org/wikipedia/commons/4/45/W3sDesign_Strategy_Design_Pattern_UML.jpg)

#### Example 6-1. Implementation Order class with pluggable discount strategies

In [1]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

class LineItem:
    
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
        
    def total(self):
        return self.price * self.quantity
    
class Order: # the Context
    
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
        
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount
    
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())
    
class Promotion(ABC): # the Strategy: an abstract base class
    
    @abstractmethod
    def discount(self, order):
        """Return discount as a positive dollar amount"""

class FidelityPromo(Promotion): # first Concrete Strategy
    """5% discount for customers with 1000 or more fidelity points"""
    
    def discount(self, order):
        return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0
    
class BulkItemPromo(Promotion): # second Concrete Strategy
    """10% discount for each LineItem with 20 or more units"""
    
    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * .1
        return discount
    
class LargeOrderPromo(Promotion): # third Concrete Strategy
    """7% discount for each LineItem with 20 or more units"""
    
    def discount(self, order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * .07
        return 0

#### Example 6-2. Sample usage of Order class with different promotions applied

In [2]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)

In [3]:
cart = [LineItem('banana', 4, .5),
       LineItem('apple', 10, 1.5),
       LineItem('watermellon', 5, 5.0)]

In [4]:
Order(joe, cart, FidelityPromo())

<Order total: 42.00 due: 42.00>

In [5]:
Order(ann, cart, FidelityPromo())

<Order total: 42.00 due: 39.90>

In [6]:
banana_cart = [LineItem('banana', 30, .5),
              LineItem('apple', 10, 1.5)]
Order(joe, banana_cart, BulkItemPromo())

<Order total: 30.00 due: 28.50>

In [7]:
long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]
Order(joe, long_order, LargeOrderPromo())

<Order total: 10.00 due: 9.30>

In [8]:
Order(joe, cart, LargeOrderPromo())

<Order total: 42.00 due: 42.00>

### Function-Oriented Strategy
전략 개체를 함수로 변경 및 Promotion 추상 클래스 제거

#### Example 6-3. Order clas with discount strategies implemented as functions

In [9]:
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

class LineItem:
    
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
        
    def total(self):
        return self.quantity * self.price
    
class Order:
    
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = cart
        self.promotion = promotion
        
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount
    
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())
    
def fidelity_promo(order):
    """5% discount for customers with 1000 or more fidelity points"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

def bulk_item_promo(order):
    """10% discount for each LineItem with 20 or more units"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount

def large_order_promo(order):
    """7% discount for orders with 10 or more distinct items"""
    distinct_items = {item.product for item in order.cart}
    return order.total() * .07 if len(distinct_items) >= 10 else 0

#### Example 6-4. Sample usage of Order class with promotions as functions

In [10]:
joe = Customer('Jone Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, .5),
       LineItem('apple', 10, 1.5),
       LineItem('watermellon', 5, 5.0)]

In [11]:
Order(joe, cart, fidelity_promo)

<Order total: 42.00 due: 42.00>

In [12]:
Order(ann, cart, fidelity_promo)

<Order total: 42.00 due: 39.90>

In [13]:
banana_cart = [LineItem('banana', 30, .5),
              LineItem('apple', 10, 1.5)]

Order(joe, banana_cart, bulk_item_promo)

<Order total: 30.00 due: 28.50>

In [14]:
long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]

Order(joe, long_order, large_order_promo)

<Order total: 10.00 due: 9.30>

In [15]:
Order(joe, cart, large_order_promo)

<Order total: 42.00 due: 42.00>

### Choosing the Best Strategy: Simple Approach

#### Example 6-5. The best_promo function applies all discounts and returns the largest

In [18]:
Order(joe, long_order, best_promo)

<Order total: 10.00 due: 9.30>

In [19]:
Order(joe, banana_cart, best_promo)

<Order total: 30.00 due: 28.50>

In [20]:
Order(ann, cart, best_promo)

<Order total: 42.00 due: 39.90>

#### Example 6-6. best_promo finds the maximum discount iterating over a list of functions

In [17]:
promos = [fidelity_promo, bulk_item_promo, large_order_promo]

def best_promo(order):
    """Select best discount available"""
    return max(promo(order) for promo in promos)

* 코드 중복 -> 버그 위험
* 새로운 전략 추가 시 promos 리스트 재선언

### Finding Strategies in a Module

globals()로 *_promo 찾기 

#### Example 6-7. The promos list is built by introspection of the module global namespace

In [21]:
promos = [globals()[name] for name in globals()
         if name.endswith('_promo') and name != 'best_promo']

def best_promo(order):
    """Select best discount available"""
    return max(promo(order) for promo in promos)

In [22]:
Order(joe, long_order, best_promo)

<Order total: 10.00 due: 9.30>

In [23]:
Order(joe, banana_cart, best_promo)

<Order total: 30.00 due: 28.50>

In [24]:
Order(ann, cart, best_promo)

<Order total: 42.00 due: 39.90>

#### Example 6-8. The promos list is built by introspection of a new promotions module

In [25]:
import inspect, promotions

promos = [func for name, func in inspect.getmembers(promotions, inspect.isfunction)]

def best_promo(order):
    """Select best discount available"""
    return max(promo(order) for promo in promos)

In [26]:
Order(joe, long_order, best_promo)

<Order total: 10.00 due: 9.30>

In [27]:
Order(joe, banana_cart, best_promo)

<Order total: 30.00 due: 28.50>

In [28]:
Order(ann, cart, best_promo)

<Order total: 42.00 due: 39.90>

## Command

[명령 디자인 패턴 UML class diagram](https://upload.wikimedia.org/wikipedia/commons/8/8e/Command_Design_Pattern_Class_Diagram.png)

#### Example 6-9. Each instacne of MacroCommand has an internal list of commands

In [29]:
class MacroCommand:
    """A command that executes a list of commands"""
    
    def __init__(self, commands):
        self.commands = list(commands)
    
    def __call__(self):
        for command in self.commands:
            command()